In [ ]:
from astropy.table import Table
from astropy.nddata import CCDData, Cutout2D
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
imgdir = "../yasone2/img_i_01"
cat = Table.read(imgdir + "/detection.cat", hdu=2)
img = CCDData.read(imgdir + "/nobkg.fits", unit="adu")
mask = CCDData.read(imgdir + "/flag.fits", unit="adu")
img_err = CCDData.read(imgdir + "/flat_fielded.weight.fits", unit="adu")
psf = CCDData.read(imgdir + "/psf.fits", unit="adu")

img_masked = img.data.copy()
img_masked[mask.data == 1] = np.nan
img_masked[mask.data >= 2 ]  = np.nan

In [ ]:
psf.data -= np.median(psf)

In [ ]:
img.data[1796, 2002]

In [ ]:
SAT_LEVEL=58_000

In [ ]:
# psf = CCDData.read("../psf_osiris_r.fits", unit="adu")

In [ ]:
global_bkg = np.median(img_err)

In [ ]:
global_bkg

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../../imaging")
import phot_utils

In [ ]:
p = plt.imshow(10000*psf.data , norm="asinh")
plt.colorbar(p)

In [ ]:
import sep

In [ ]:
radii = np.linspace(0.1, 100, 100)
fluxes = sep.sum_circle(phot_utils.swap_byteorder(psf.data), [(psf.shape[0]-1)/2], [(psf.shape[1]-1)/2], radii)[0]

In [ ]:
def calculate_cumulative_profile(psf_model, radii, center=None, 
                                  grid_size=201, pixel_scale=1.0,
                                  normalize=True):
    """
    Calculate the cumulative flux profile (curve of growth) for an analytic PSF.
    
    Parameters
    ----------
    psf_model : astropy.modeling.Model
        Analytic PSF model (e.g., Gaussian2D, Moffat2D).
        Should be centered at (0, 0) or specify center parameter.
    radii : array-like
        Array of aperture radii (in pixels) at which to calculate enclosed flux.
    center : tuple of float, optional
        Center position (x, y) of the PSF. If None, uses grid center.
    grid_size : int, optional
        Size of the grid to evaluate PSF on. Should be odd. Default is 201.
    pixel_scale : float, optional
        Pixel scale for the grid. Default is 1.0.
    normalize : bool, optional
        If True, normalize profile to total flux = 1. Default is True.
    
    Returns
    -------
    result : dict
        Dictionary containing:
        - 'radii': Array of aperture radii
        - 'enclosed_flux': Cumulative flux enclosed within each radius
        - 'fractional_flux': Fraction of total flux (if normalize=True)
        - 'total_flux': Total integrated flux from the PSF
        - 'psf_image': Evaluated PSF on the grid
    """
    
    # Create coordinate grid
    y, x = np.mgrid[0:grid_size, 0:grid_size] * pixel_scale
    
    # Determine center
    if center is None:
        center = (grid_size * pixel_scale / 2, grid_size * pixel_scale / 2)
    
    # Evaluate PSF on grid
    psf_image = psf_model(x-center[0], y - center[0])
    
    # Calculate total flux
    total_flux = np.sum(psf_image) * pixel_scale**2
    
    # Calculate enclosed flux for each radius
    enclosed_flux = np.zeros_like(radii, dtype=float)
    
    for i, radius in enumerate(radii):
        # Create circular aperture
        aperture = CircularAperture(center, r=radius)
        
        # Perform aperture photometry
        phot_table = aperture_photometry(psf_image, aperture)
        enclosed_flux[i] = phot_table['aperture_sum'][0]
    
    # Calculate fractional flux
    if normalize and total_flux > 0:
        fractional_flux = enclosed_flux / total_flux
    else:
        fractional_flux = enclosed_flux
    
    return {
        'radii': np.array(radii),
        'enclosed_flux': enclosed_flux,
        'fractional_flux': fractional_flux,
        'total_flux': total_flux,
        'psf_image': psf_image,
        'center': center
    }


In [ ]:
from astropy.modeling.models import Gaussian2D, Moffat2D
from photutils.aperture import CircularAperture, aperture_photometry


In [ ]:
?Moffat2D

In [ ]:
sigma = 1.6
gaussian_psf = Gaussian2D(amplitude=1/(2*np.pi*sigma**2), 
                          x_mean=0, y_mean=0,
                          x_stddev=sigma, y_stddev=sigma)

# Calculate numerical profile
gauss_result = calculate_cumulative_profile(gaussian_psf, radii,
                                             grid_size=201)


moff_psf = Moffat2D(amplitude=0.05, 
                          gamma=2.1, alpha=2.6)

# Calculate numerical profile
moff_result = calculate_cumulative_profile(moff_psf, radii, pixel_scale=1,
                                             grid_size=201)


In [ ]:

def calculate_analytical_gaussian_profile(radii, sigma, A=1):
    """
    Calculate the analytical cumulative profile for a 2D Gaussian PSF.
    
    For a 2D Gaussian with standard deviation sigma, the enclosed flux
    within radius r is: 1 - exp(-r^2 / (2*sigma^2))
    
    Parameters
    ----------
    sigma : float
        Standard deviation of the Gaussian (assumes circular, sigma_x = sigma_y).
    radii : array-like
        Array of aperture radii.
    
    Returns
    -------
    fractional_flux : array
        Fraction of total flux enclosed within each radius.
    """
    radii = np.array(radii)
    return A * (1 - np.exp(-radii**2 / (2 * sigma**2)))

In [ ]:
def calculate_analytical_moffat_profile(radii, alpha, beta, A=1):
    """
    Calculate the analytical cumulative profile for a Moffat PSF.
    
    For a Moffat profile with parameters alpha and beta, the enclosed flux
    within radius r is: 1 - (1 + (r/alpha)^2)^(1-beta)
    
    Parameters
    ----------
    alpha : float
        Core width parameter of the Moffat profile.
    beta : float
        Power law index of the Moffat profile.
    radii : array-like
        Array of aperture radii.
    
    Returns
    -------
    fractional_flux : array
        Fraction of total flux enclosed within each radius.
    """
    radii = np.array(radii)
    flux = 1 - (1 + (radii / alpha)**2)**(1 - beta)

    return flux  * A


In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def calculate_analytical_both_profile(radii, alpha, beta, A=1, sigma=1, B=0.5):
    return calculate_analytical_moffat_profile(radii, alpha=alpha, beta=beta, A=A) + calculate_analytical_gaussian_profile(radii, sigma=sigma, A=B)


In [ ]:
popt_g, covt_g = curve_fit(calculate_analytical_gaussian_profile, radii, fluxes / fluxes[-1])
popt_g, np.sqrt(np.diag(covt_g))

In [ ]:
popt_b, covt_b = curve_fit(calculate_analytical_both_profile, radii, fluxes / fluxes[-1], p0=[2.6, 2.1, 0.02, 3, 0.02])
popt_b, np.sqrt(np.diag(covt_b))

In [ ]:
popt, covt = curve_fit(calculate_analytical_moffat_profile, radii, fluxes / fluxes[-1], p0=[2.6, 2.1, 0.02])

In [ ]:
popt

In [ ]:
popt, np.sqrt(np.diag(covt))

In [ ]:
popt_b

In [ ]:
def psf_model_1d(radii):
    return calculate_analytical_both_profile(radii, *popt_b)

In [ ]:
def psf_model_2d(x, y):
    return psf_model_1d(np.sqrt(x**2 + y**2))

In [ ]:
import astropy

In [ ]:
gamma_moff, alpha_moff, A_moff, sigma_gauss, A_gauss = popt_b

psf_gauss_best = Gaussian2D(amplitude=A_gauss/(2*np.pi*sigma_gauss**2), x_stddev=sigma_gauss, y_stddev=sigma_gauss)
moff_norm = (alpha_moff - 1) / (np.pi * gamma_moff**2)
psf_moff_best = Moffat2D(amplitude=A_moff * moff_norm, alpha=alpha_moff, gamma=gamma_moff)

psf_model_2d = psf_gauss_best + psf_moff_best

In [ ]:
plt.plot(radii, fluxes / fluxes[-1])
plt.plot(radii, gauss_result['enclosed_flux'] / gauss_result['enclosed_flux'][-1])
# plt.plot(radii, moff_result['enclosed_flux'] / moff_result['enclosed_flux'][-1])


y = calculate_analytical_moffat_profile(radii=radii, alpha=popt[0], beta=popt[1], A=popt[2])
plt.plot(radii, y)


# y = calculate_analytical_moffat_profile(radii=radii, alpha=gamma_moff, beta=alpha_moff, A=A_moff )
# plt.plot(radii, y)
# both = calculate_cumulative_profile(psf_model_2d, radii,
#                                              grid_size=520)

# plt.plot(radii, both['enclosed_flux'] )

# y = calculate_analytical_both_profile(radii, *popt_b)
# plt.plot(radii, y)

plt.xscale("log")

In [ ]:
import warnings

In [ ]:
from scipy.optimize import minimize


In [ ]:
def fit_model_psf_chi2(image, psf, uncertainty=None, mask=None, 
                 initial_scale=1.0, initial_shift=(0.0, 0.0),
                 bounds = None,
                 method='Nelder-Mead'):
    """
    Fit a PSF model to an image cutout by minimizing chi-squared.
    Optimizes only the shift (x, y) and amplitude scale of the PSF.
    
    Parameters
    ----------
    image : 2D array
        Image cutout to fit. Can contain NaN values.
    psf : 2D array
        PSF model to fit to the image.
    uncertainty : 2D array, optional
        Uncertainty/error map for the image. If None, assumes uniform weights.
    mask : 2D array of bool, optional
        Boolean mask where True indicates pixels to exclude from fit.
        If None, only NaN pixels are masked.
    initial_scale : float, optional
        Initial guess for PSF amplitude scale. Default is 1.0.
    initial_shift : tuple of float, optional
        Initial guess for PSF shift (dy, dx) in pixels. Default is (0, 0).
    method : str, optional
        Optimization method for scipy.optimize.minimize. Default is 'Nelder-Mead'.
    
    Returns
    -------
    result : dict
        Dictionary containing:
        - 'scale': Optimal amplitude scale factor
        - 'shift': Optimal shift (dy, dx) in pixels
        - 'chi2': Minimum chi-squared value
        - 'reduced_chi2': Reduced chi-squared (chi2 / dof)
        - 'model': Best-fit PSF model image
        - 'residual': Residual image (data - model)
        - 'success': Whether optimization converged
    """
    
    # Handle NaN pixels in image
    image_clean = np.copy(image)
    nan_mask = np.isfinite(image_clean)
    
    # Combine NaN mask with user-provided mask
    if mask is None:
        combined_mask = nan_mask
    else:
        combined_mask = nan_mask | mask
    
    # Set up uncertainty
    if uncertainty is None:
        uncertainty = 1.0

    
    # Calculate valid pixel count for degrees of freedom
    n_valid = np.sum(combined_mask)
    n_params = 4  # scale, dy, dx
    dof = max(1, n_valid - n_params)
    nX, nY = image.shape

    x_img, y_img = np.meshgrid(np.arange(nY), np.arange(nX))
    x_c, y_c = (nY - 1)/2, (nX-1)/2



    if bounds is None:
        bounds = (-nY/4, nY/4), (-nX/4, nX/4)
    def chi2_func(params):
        """Calculate chi-squared for given parameters."""
        scale, dx, dy, bg = params

        # Shift the PSF
        shifted_psf = psf(x_img - dx - x_c, y_img - dy - y_c)

        
        # Scale the PSF
        model = np.minimum(scale * shifted_psf + bg, SAT_LEVEL)
        
        # Calculate residuals only for valid pixels
        residual = image_clean - model
        
        # Calculate chi-squared
        chi2 = np.sum(np.abs(residual[combined_mask])**2)
        
        return chi2
    
    # Initial parameters
    x0 = [initial_scale, initial_shift[0], initial_shift[1], 0]
    
    # Optimize
    all_bounds = ((0, None), bounds[0], bounds[1], (-100, 100))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        opt_result = minimize(chi2_func, x0, method=method, bounds=all_bounds)
    
    # Extract best-fit parameters
    best_scale, best_dx, best_dy, best_bg = opt_result.x
    best_shift = (best_dx, best_dy)
    
    # Generate best-fit model
    shifted_psf = psf(x_img - best_dx - x_c, y_img - best_dy - y_c)
    best_model = np.minimum(best_scale * shifted_psf + best_bg, SAT_LEVEL)
    
    # Calculate final residual
    residual = image_clean - best_model
    
    # Calculate final chi-squared
    min_chi2 = opt_result.fun
    reduced_chi2 = min_chi2 / dof
    
    return {
        'scale': best_scale,
        'shift': best_shift,
        'bkg': best_bg,
        'chi2': min_chi2,
        'reduced_chi2': reduced_chi2,
        'model': best_model,
        'residual': residual,
        'success': opt_result.success,
        'n_valid_pixels': n_valid,
        'dof': dof
    }


In [ ]:
from scipy.ndimage import shift as nd_shift


In [ ]:
def fit_psf_chi2(image, psf, uncertainty=None, mask=None, 
                 initial_scale=1.0, initial_shift=(0.0, 0.0),
                 bounds = None,
                 method='Nelder-Mead'):
    """
    Fit a PSF model to an image cutout by minimizing chi-squared.
    Optimizes only the shift (x, y) and amplitude scale of the PSF.
    
    Parameters
    ----------
    image : 2D array
        Image cutout to fit. Can contain NaN values.
    psf : 2D array
        PSF model to fit to the image.
    uncertainty : 2D array, optional
        Uncertainty/error map for the image. If None, assumes uniform weights.
    mask : 2D array of bool, optional
        Boolean mask where True indicates pixels to exclude from fit.
        If None, only NaN pixels are masked.
    initial_scale : float, optional
        Initial guess for PSF amplitude scale. Default is 1.0.
    initial_shift : tuple of float, optional
        Initial guess for PSF shift (dy, dx) in pixels. Default is (0, 0).
    method : str, optional
        Optimization method for scipy.optimize.minimize. Default is 'Nelder-Mead'.
    
    Returns
    -------
    result : dict
        Dictionary containing:
        - 'scale': Optimal amplitude scale factor
        - 'shift': Optimal shift (dy, dx) in pixels
        - 'chi2': Minimum chi-squared value
        - 'reduced_chi2': Reduced chi-squared (chi2 / dof)
        - 'model': Best-fit PSF model image
        - 'residual': Residual image (data - model)
        - 'success': Whether optimization converged
    """
    
    # Handle NaN pixels in image
    image_clean = np.copy(image)
    nan_mask = np.isfinite(image_clean)
    
    # Combine NaN mask with user-provided mask
    if mask is None:
        combined_mask = nan_mask
    else:
        combined_mask = nan_mask | mask
    
    # Set up uncertainty
    if uncertainty is None:
        uncertainty = 1.0

    
    # Calculate valid pixel count for degrees of freedom
    n_valid = np.sum(combined_mask)
    n_params = 4  # scale, dy, dx
    dof = max(1, n_valid - n_params)
    nX, nY = image.shape

    psf_padded = pad_psf_to_image(psf, image.shape)

    if bounds is None:
        bounds = (-nY/4, nY/4), (-nX/4, nX/4)
    def chi2_func(params):
        """Calculate chi-squared for given parameters."""
        scale, dx, dy, bg = params

        # Shift the PSF
        shifted_psf = nd_shift(psf_padded, shift=(dx, dy), order=3, mode='constant', cval=0)

        
        # Scale the PSF
        model = np.minimum(scale * shifted_psf + bg, SAT_LEVEL)
        
        # Calculate residuals only for valid pixels
        residual = image_clean - model
        
        # Calculate chi-squared
        chi2 = np.sum(np.abs(residual[combined_mask])**2)
        
        return chi2
    
    # Initial parameters
    x0 = [initial_scale, initial_shift[0], initial_shift[1], 0]
    
    # Optimize
    all_bounds = ((0, None), bounds[0], bounds[1], (-100, 100))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        opt_result = minimize(chi2_func, x0, method=method, bounds=all_bounds)
    
    # Extract best-fit parameters
    best_scale, best_dx, best_dy, best_bg = opt_result.x
    best_shift = (best_dx, best_dy)
    
    # Generate best-fit model
    shifted_psf = nd_shift(psf_padded, shift=(best_dx, best_dy), order=3, mode='constant', cval=0)
    best_model = np.minimum(best_scale * shifted_psf + best_bg, SAT_LEVEL)
    
    # Calculate final residual
    residual = image_clean - best_model
    
    # Calculate final chi-squared
    min_chi2 = opt_result.fun
    reduced_chi2 = min_chi2 / dof
    
    return {
        'scale': best_scale,
        'shift': best_shift,
        'bkg': best_bg,
        'chi2': min_chi2,
        'reduced_chi2': reduced_chi2,
        'model': best_model,
        'residual': residual,
        'success': opt_result.success,
        'n_valid_pixels': n_valid,
        'dof': dof
    }


In [ ]:
2901/2

In [ ]:
psf_model_2d(0,0)

In [ ]:
fit = fit_model_psf_chi2(psf[1400:1501, 1400:1501], psf_model_2d, initial_scale=1e10, )
fig, axs = plt.subplots(1, 2)
plt.sca(axs[0])
p = plt.imshow(fit['model'],)
plt.colorbar(p)

plt.sca(axs[1])
p = plt.imshow(fit['residual'],  cmap="RdBu")
plt.colorbar(p)
fit

In [ ]:
x, y = np.meshgrid(np.arange(-24.5, 25), np.arange(-24.5, 25))

In [ ]:
p = plt.imshow(psf_model_2d(x, y))
plt.colorbar(p)

In [ ]:
model = fit["scale"]*psf_model_2d(x - fit["shift"][0], y - fit["shift"][1])
print(np.sum((psf - model)**2))
p = plt.imshow((psf - model) / np.max(psf))

plt.colorbar(p, label="residual / max value")
plt.show()
p = plt.imshow(np.log10(psf / model), vmin=-0.3, vmax=0.3, cmap="RdBu")
plt.colorbar(p, label="log residual")

In [ ]:
plt.imshow(psf_model(x, y), norm="log")

# Fitting psf to individual stars

In [ ]:
plt.imshow(img_masked)

In [ ]:
def get_cutouts(
    ccd,
    cat
):
    cutouts = []
    for i in range(len(cat)):
        x0, y0 = round(cat["XWIN_IMAGE"][i])-1,  round(cat["YWIN_IMAGE"][i])-1
        R = round(4*cat["FLUX_RADIUS"][i])
        xmin = max(x0 - R, 0)
        xmax = min(x0 + R, ccd.shape[0]-1)
        ymin = max(y0 - R, 0)
        ymax = min(y0 + R, ccd.shape[1]-1)
        cutout = ccd[ymin:ymax, xmin:xmax]
        cutouts.append(cutout)

    return cutouts

In [ ]:
cat_bright = cat[(cat["FLAGS"] & 4 > 0) | (cat["IMAFLAGS_ISO"] &1 > 0)]

In [ ]:
plt.hist(cat_bright["MAG_APER"][:, 4], np.linspace(-20, -5, 20))
plt.hist(cat["MAG_APER"][:, 4], np.linspace(-20, -5, 20), histtype="step")

In [ ]:
cutouts = get_cutouts(img_masked, cat_bright)

In [ ]:
cutouts_good = [cutout for cutout in cutouts if np.mean(np.isfinite(cutout)) > 0.5]

In [ ]:
fwhm = 0.65

In [ ]:
plt.imshow(cutouts_good[5])

In [ ]:
for cutout in cutouts_good[20:40]:
    plt.imshow(cutout)
    plt.show()

In [ ]:
def pad_psf_to_image(psf, image_shape, psf_center=None, fill_value=0):
    """
    Pad a PSF to match the size of an image, centering it appropriately.
    
    Parameters
    ----------
    psf : 2D array
        The PSF model to pad.
    image_shape : tuple
        Shape of the target image (ny, nx).
    psf_center : tuple of float, optional
        Center position of the PSF in the original PSF array (y, x).
        If None, assumes PSF is centered at its geometric center.
    fill_value : float, optional
        Value to use for padding. Default is 0.
    
    Returns
    -------
    padded_psf : 2D array
        PSF padded to match image_shape, with the PSF center positioned
        at the center of the padded array.
    
    Examples
    --------
    >>> # Pad a 15x15 PSF to match a 25x25 image
    >>> small_psf = create_small_psf()  # shape (15, 15)
    >>> padded = pad_psf_to_image(small_psf, (25, 25))
    >>> padded.shape
    (25, 25)
    """
    psf_shape = psf.shape
    target_shape = image_shape
    
    # Determine PSF center
    if psf_center is None:
        psf_center = ((psf_shape[0] - 1) / 2, (psf_shape[1] - 1) / 2)
    
    # Calculate target center
    target_center = ((target_shape[0] - 1) / 2, (target_shape[1] - 1) / 2)
    
    # Calculate padding needed
    # We want psf_center to align with target_center
    pad_before_y = int(np.floor(target_center[0] - psf_center[0]))
    pad_before_x = int(np.floor(target_center[1] - psf_center[1]))
    
    pad_after_y = target_shape[0] - psf_shape[0] - pad_before_y
    pad_after_x = target_shape[1] - psf_shape[1] - pad_before_x
    
    # Handle cases where PSF is larger than target
    if pad_before_y < 0 or pad_after_y < 0 or pad_before_x < 0 or pad_after_x < 0:
        # Crop PSF instead of padding
        crop_start_y = max(0, -pad_before_y)
        crop_end_y = psf_shape[0] + min(0, pad_after_y)
        crop_start_x = max(0, -pad_before_x)
        crop_end_x = psf_shape[1] + min(0, pad_after_x)
        
        cropped_psf = psf[crop_start_y:crop_end_y, crop_start_x:crop_end_x]
        
        # Now pad the cropped PSF
        pad_before_y = max(0, pad_before_y)
        pad_before_x = max(0, pad_before_x)
        pad_after_y = max(0, pad_after_y)
        pad_after_x = max(0, pad_after_x)
        
        padded_psf = np.pad(cropped_psf, 
                           ((pad_before_y, pad_after_y), 
                            (pad_before_x, pad_after_x)),
                           mode='constant', constant_values=fill_value)
    else:
        # Simple padding case
        padded_psf = np.pad(psf, 
                           ((pad_before_y, pad_after_y), 
                            (pad_before_x, pad_after_x)),
                           mode='constant', constant_values=fill_value)
    
    return padded_psf

In [ ]:
psf_corr = psf - np.median(psf)

In [ ]:
p = plt.imshow(psf_corr / np.max(psf_corr), vmin=-0.001, vmax=0.001, cmap="RdBu")
plt.colorbar()

In [ ]:
plt.hist(psf.data.flatten() / np.max(psf), bins=np.linspace(-0.002, 0.004, 100));
plt.yscale("log")

plt.axvline(np.median(psf.data / np.max(psf)), color="orange")

In [ ]:
for img in cutouts_good[20::10]:
    fig, axs = plt.subplots(2, 3, layout="constrained", figsize=(6, 4))
    plt.sca(axs[0][0])
    vmin, vmax = -10, 10
    
    img_kwargs = dict(origin="lower", vmin=vmin, vmax=vmax, cmap="RdBu", norm="asinh")
    plt.imshow(img/global_bkg, **img_kwargs)
    plt.title("image")

    psf_fit = fit_model_psf_chi2(img, psf_model_2d, initial_scale=100_000)

    if True: #psf_fit["success"]:
    
        plt.sca(axs[0][1])
        plt.imshow(psf_fit["model"]/global_bkg, **img_kwargs)
        plt.title("ana psf model")
    
        plt.sca(axs[0][2])
        p = plt.imshow(psf_fit["residual"]/global_bkg, **img_kwargs)
        plt.title("ana psf residual")

    psf_fit = fit_psf_chi2(img, psf_corr, initial_scale=100_000)

    if True: #psf_fit["success"]:
    
        plt.sca(axs[1][1])
        plt.imshow(psf_fit["model"]/global_bkg, **img_kwargs)
        plt.title("stacked psf model")
    
        plt.sca(axs[1][2])
        p = plt.imshow(psf_fit["residual"]/global_bkg, **img_kwargs)
        plt.title("stacked psf residual")

    axs[1][0].remove()

    fig.colorbar(p, ax=axs[0][-1], shrink=0.8, label="count / bkg rms")